In [3]:
if (!require(rvest)) install.packages('rvest')
library(rvest)

if (!require(tidyverse)) install.packages('tidyverse')
library(tidyverse)

In [4]:
getLinks <- function(termGbn){
    base_url <- 'http://gaonchart.co.kr/main/section/chart/online.gaon?nationGbn=T&serviceGbn=ALL'
    sample_url <- paste0(base_url, '&termGbn=', termGbn)
    dates <-     
        read_html(sample_url) %>%
        html_nodes('div[class=fr]') %>%
        html_nodes('select') %>%
        html_nodes('option') %>%
        html_attr('value') %>%
        str_subset(pattern = '^[0-9]{6}$')
    
    hitYear <-
        dates %>%
        substr(1,4) # 연도 ex) 2022 => 2022년

    targetTime <-
        dates %>%
        substr(5,6) # 몇 주차 또는 몇 월차인지 ex) 14 => 14주차
    
    completed_url <- paste0(base_url, '&targetTime=', targetTime, '&hitYear=', hitYear, '&termGbn=', termGbn)
    return(completed_url)
}

In [5]:
weekLinks <- getLinks('week')

In [6]:
target_url <- weekLinks[422]
target_url

[1] "http://gaonchart.co.kr/main/section/chart/online.gaon?nationGbn=T&serviceGbn=ALL&targetTime=18&hitYear=2014&termGbn=week"

In [7]:
year_week <- 
    target_url %>%
    strsplit('=|&') %>%
    unlist() %>%
    str_subset('^[0-9]{2,4}$') %>%
    rev() %>%
    paste0(collapse = '')
year_week

[1] "201418"

In [22]:
html_chart <- read_html(target_url) %>%
    html_nodes('div[class=chart]')

td <- html_chart %>% 
    html_nodes('td') %>%
    html_text() %>%
    str_split('\\n|\\||\\r|\\t') %>%
    unlist() %>%
    str_subset('^$', negate = TRUE)

change_grp <- html_chart %>%
    html_nodes('td[class=change]') %>%
    html_nodes('span') %>%
    html_attr('class')

In [45]:
index_PLAY <- str_which(td, 'PLAY')
PLAY_ZONE <- c(11, 12)
index_TRUE_PLAY <- index_PLAY[index_PLAY %in% PLAY_ZONE][1]
index_TRUE_PLAY

[1] 11

In [49]:
index_PLAY[index_PLAY %in% PLAY_ZONE][1]

[1] 11

In [40]:
chart_piece <- td %>% 
    matrix(ncol = index_TRUE_PLAY, byrow = TRUE) %>%
    as_tibble()
chart_piece %>% head()

V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,V11
<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
1,1,200%,AKMU (악뮤),PLAY,YG Entertainment,지니뮤직,트위터,페이스북,웨이브,PLAY
2,1,봄 사랑 벚꽃 말고,"HIGH4, 아이유 (IU)",봄 사랑 벚꽃 말고,엔에이피엔터테인먼트,Kakao Entertainment,트위터,페이스북,웨이브,PLAY
3,2,티가 나나봐,"15& (박지민, 백예린)",티가 나나봐,JYP Entertainment,지니뮤직,트위터,페이스북,웨이브,PLAY
4,2,Mr. Chu (On Stage),에이핑크 (Apink),Pink Blossom,플레이엠엔터테인먼트,Kakao Entertainment,트위터,페이스북,웨이브,PLAY
5,-,Give Love,AKMU (악뮤),PLAY,YG Entertainment,지니뮤직,트위터,페이스북,웨이브,PLAY
6,1,인연,"윤민수, 신용재",불후의 명곡 - 전설을 노래하다 (이선희 2편),KBS미디어,Kakao Entertainment,트위터,페이스북,웨이브,PLAY


In [41]:
chart_piece <- td %>% 
    matrix(ncol = index_TRUE_PLAY, byrow = TRUE) %>%
    as.tibble()

In [42]:
# Note : ifelse always returns an object of the same length as the condition. so use if/else for this case
chart <- data.frame(year_week = year_week,
                    ranking = chart_piece$V1,
                    change_grp = change_grp,
                    change_val = chart_piece$V2,
                    title = chart_piece$V3,
                    artist = chart_piece$V4,
                    gaon_index = if (ncol(chart_piece) == 12) chart_piece$V6 else NA,  
                    production = if(ncol(chart_piece) == 12) chart_piece$V7 else chart_piece$V6,
                    distribution = if(ncol(chart_piece) == 12) chart_piece$V8 else chart_piece$V7)
chart

year_week,ranking,change_grp,change_val,title,artist,gaon_index,production,distribution
<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<lgl>,<chr>,<chr>
201418,1,up,1,200%,AKMU (악뮤),NA,YG Entertainment,지니뮤직
201418,2,up,1,봄 사랑 벚꽃 말고,"HIGH4, 아이유 (IU)",NA,엔에이피엔터테인먼트,Kakao Entertainment
201418,3,down,2,티가 나나봐,"15& (박지민, 백예린)",NA,JYP Entertainment,지니뮤직
201418,4,up,2,Mr. Chu (On Stage),에이핑크 (Apink),NA,플레이엠엔터테인먼트,Kakao Entertainment
201418,5,NA,-,Give Love,AKMU (악뮤),NA,YG Entertainment,지니뮤직
201418,6,up,1,인연,"윤민수, 신용재",NA,KBS미디어,Kakao Entertainment
201418,7,up,2,야생화,박효신,NA,젤리피쉬 엔터테인먼트,Stone Music Entertainment
201418,8,up,4,그 중에 그대를 만나,이선희,NA,후크엔터테인먼트,Kakao Entertainment
201418,9,down,1,얼음들,AKMU (악뮤),NA,YG Entertainment,지니뮤직
